In [ ]:
!pip install pandas

import pandas as pd
import numpy as np

In [ ]:
# Import dataset
df = pd.read_csv('https://raw.githubusercontent.com/syha-studio/FinalProjectDataMining/refs/heads/main/Indeks%20Standar%20Pencemaran%20Udara%20(ISPU)%20Tahun%202021%20-%202022.xlsx%20-%20Fixed%20Data.csv')
print(df.head())

   Periode Data    Tanggal Stasiun  PM10  PM2,5   SO2    CO    O3   NO2 MAX  \
0        202101  1/22/2021    DKI5  45.0    NaN  21.0  13.0  40.0  15.0  45   
1        202101  1/23/2021    DKI5  80.0    NaN  22.0  44.0  44.0  22.0  80   
2        202101  1/24/2021    DKI5  27.0    NaN  14.0   9.0  29.0   NaN  29   
3        202101  1/25/2021    DKI5  55.0    NaN  19.0  29.0  67.0  13.0  67   
4        202101  1/26/2021    DKI5  57.0    NaN  21.0  23.0  36.0  16.0  57   

  Critical Category  
0     PM10     BAIK  
1     PM10   SEDANG  
2       CO     BAIK  
3       CO   SEDANG  
4     PM10   SEDANG  


In [ ]:
# Data cleaning
df = df.drop(columns=['Periode Data', 'Tanggal', 'Stasiun', 'MAX', 'Critical'])
df = df[df['Category'] != "TIDAK ADA DATA"]
df = df.dropna(subset=['Category'])

In [ ]:
# Imputation
df['PM10'] = df.groupby('Category')['PM10'].transform(lambda x: x.fillna(x.median()))
df['PM2,5'] = df.groupby('Category')['PM2,5'].transform(lambda x: x.fillna(x.median()))
df['SO2'] = df.groupby('Category')['SO2'].transform(lambda x: x.fillna(x.median()))
df['CO'] = df.groupby('Category')['CO'].transform(lambda x: x.fillna(x.median()))
df['O3'] = df.groupby('Category')['O3'].transform(lambda x: x.fillna(x.median()))
df['NO2'] = df.groupby('Category')['NO2'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# Encode categories
Category = {"TIDAK SEHAT": 1, "SEDANG": 2, "BAIK": 3}
df["Category"] = df["Category"].map(Category)

In [ ]:
# Split data into features and labels
X = df[['PM10', 'PM2,5', 'SO2', 'CO', 'O3', 'NO2']]
y = df['Category']

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27)

# Train model
from sklearn.ensemble import RandomForestClassifier
modelfix = RandomForestClassifier()
modelfix.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# mem-pickle-kan model

import pickle
# nama pickle - classifier.pkl
with open("modelfix.pkl", "wb") as pickle_out:
    # nama model - classifier
    pickle.dump(modelfix, pickle_out)

In [ ]:
%%writefile app.py
import pandas as pd
import numpy as np
import pickle
import streamlit as st
from PIL import Image

# Load model yang digunakan untuk melakukan prediksi
with open('modelfix.pkl', 'rb') as pickle_in:
    model = pickle.load(pickle_in)

def prediction(pm10, pm25, so2, co, o3, no2):
    # Konversi input menjadi float
    input_data = [[float(pm10), float(pm25), float(so2), float(co), float(o3), float(no2)]]

    # Melakukan prediksi
    prediction = model.predict(input_data)

    # Menyusun kategori berdasarkan hasil prediksi
    if prediction[0] == 1:
        return "Tidak Sehat"
    elif prediction[0] == 2:
        return "Sedang"
    else:
        return "Baik"

def main():
    # Judul untuk web
    st.title("Prediksi Kualitas Udara")

    # Menentukan gaya web, latar belakang, h1
    html_temp = """
    <div style="background-color:lightblue;padding:13px">
    <h1 style="color:black;text-align:center;">Streamlit Air Quality Prediction ML App</h1>
    </div>
    """

    # Menampilkan script yang disimpan di variabel html_temp di frontend pengguna
    st.markdown(html_temp, unsafe_allow_html=True)

    # Membuat input numerik yang akan digunakan untuk prediksi
    pm10 = st.number_input("PM10", min_value=0.0, format="%.2f")
    pm25 = st.number_input("PM2.5", min_value=0.0, format="%.2f")
    so2 = st.number_input("SO2", min_value=0.0, format="%.2f")
    co = st.number_input("CO", min_value=0.0, format="%.2f")
    o3 = st.number_input("O3", min_value=0.0, format="%.2f")
    no2 = st.number_input("NO2", min_value=0.0, format="%.2f")
    result = ""

    # Jika tombol prediksi ditekan
    if st.button("Prediksi"):
        try:
            result = prediction(pm10, pm25, so2, co, o3, no2)
            st.success('Prediksi Kualitas Udara: {}'.format(result))
        except Exception as e:
            st.error(f"Terjadi kesalahan: {str(e)}")

if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.118.241.224


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.118.241.224:8501

⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://olive-mirrors-shave.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
